# 3. SQL para preparación de datos

# 3.1 Introducción
Según Forbes, los analistas de datos pasan casi el 80% de su tiempo preparando datos. Trabajar con datos mal organizados puede resultar en análisis incorrectos. Aquí es donde SQL se vuelve una herramienta poderosa, ayudándote a ordenar y limpiar los datos eficientemente.

En este capítulo, aprenderás a juntar datos de diferentes fuentes usando las funciones JOIN y UNION de SQL. También conocerás herramientas que te ayudarán a limpiar los datos, como las funciones CASE WHEN, COALESCE, NULLIF y LEAST/GREATEST. Y para terminar, veremos cómo evitar información duplicada usando el comando DISTINCT.

Todo esto te permitirá trabajar con datos más limpios y precisos.

In [ ]:
# @title Preparación del entorno
!pip install tabulate
import sqlite3
from tabulate import tabulate

# Descargamos la base de datos sqlda
!curl https://raw.githubusercontent.com/limspiga/data-modeling/main/db/sqlda.sql -O

# Código auxiliar. Nota: ejecutar cuando se carge este libro.
def sql_exec_query(query):
  conn = sqlite3.connect('sqlda.sql')
  cur = conn.cursor()
  try:
    cur.execute(query)
    headers = [column[0] for column in cur.description]
    print(tabulate(cur,  headers=headers))
    # conn.commit()
  except Exception as e:
    print(str(e))
    # conn.rollback()
  cur.close()
  conn.close()

def sql_exec(query):
  conn = sqlite3.connect('sqlda.sql')
  cur = conn.cursor()
  try:
      cur.execute(query)
      conn.commit()
      print("Query executed successfully")
  except sqlite3.Error as e:
      print(str(e))
      conn.rollback()
  cur.close()
  conn.close()

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 60.5M  100 60.5M    0     0  76.9M      0 --:--:-- --:--:-- --:--:-- 76.9M


# 3.2 Ensamble de Datos

En el segundo capítulo abordaste operaciones básicas en SQL para el manejo de una sola tabla. En esta sección, aprenderás a trabajar con datos distribuidos en múltiples tablas, utilizando uniones y agrupaciones para consolidar la información.

## 3.2.1  Conexión de Tablas con JOIN
En muchas situaciones, la información que queremos no está toda en una sola tabla. Usar un simple comando SELECT en una tabla no es suficiente. Pero no hay de qué preocuparse, porque SQL nos da herramientas para unir tablas relacionadas, como el comando JOIN.

Para que lo veas más claro, imagina que tenemos dos tablas en la base de datos ZoomZoom:

- una de concesionarios y otra de
- vendedores.

In [ ]:
sql_exec_query('''
  PRAGMA table_info(dealerships);
''')

# Figura 3.1: Estructura de la tabla de concesionarios

  cid  name            type        notnull  dflt_value      pk
-----  --------------  --------  ---------  ------------  ----
    0  dealership_id   INTEGER           0                   1
    1  street_address  TEXT              0                   0
    2  city            TEXT              0                   0
    3  state           TEXT              0                   0
    4  postal_code     TEXT              0                   0
    5  latitude        REAL              0                   0
    6  longitude       REAL              0                   0
    7  date_opened     DATETIME          0                   0
    8  date_closed     DATETIME          0                   0


Y la tabla de los vendedores se ve así:

In [ ]:
sql_exec_query('''
  PRAGMA table_info(salespeople);
''')

# Figura 3.2: Estructura de la tabla de vendedores

  cid  name              type        notnull  dflt_value      pk
-----  ----------------  --------  ---------  ------------  ----
    0  salesperson_id    INTEGER           0                   1
    1  dealership_id     INTEGER           0                   0
    2  title             TEXT              0                   0
    3  first_name        TEXT              0                   0
    4  last_name         TEXT              0                   0
    5  suffix            TEXT              0                   0
    6  username          TEXT              0                   0
    7  gender            TEXT              0                   0
    8  hire_date         DATETIME          0                   0
    9  termination_date  DATETIME          0                   0


En la tabla que muestra los detalles de los vendedores, hay una columna etiquetada como `dealership_id`. Esta columna tiene un papel muy importante: nos dice a qué concesionario pertenece cada vendedor, conectándolo directamente con una entrada específica en la tabla de concesionarios a través de una coincidencia de `dealership_id`.

Esta relación es posible gracias a un concepto llamado "**clave extranjera**", que en este caso es el `dealership_id` en la tabla de vendedores. Una "clave extranjera" es simplemente un campo que establece una conexión directa con el campo correspondiente (la "clave principal") en otra tabla, creando así un puente entre las dos tablas.

Gracias a este puente, podemos explorar y analizar los datos de formas muy variadas e interesantes. Imagina que quieres saber quiénes son los vendedores que trabajan en los concesionarios de California; este tipo de preguntas se pueden responder fácilmente. Primero, identificarías qué concesionarios están en California y luego usarías esa información para encontrar a los vendedores correspondientes. A continuación, veremos cómo hacerlo con una consulta SQL:


In [ ]:
sql_exec_query('''
SELECT *
FROM dealerships
WHERE state='CA';
''')

# Figura 3.3: Concesionarios en California

  dealership_id  street_address              city         state      postal_code    latitude    longitude  date_opened          date_closed
---------------  --------------------------  -----------  -------  -------------  ----------  -----------  -------------------  -------------
              2  808 South Hobart Boulevard  Los Angeles  CA               90005     34.0578     -118.305  2017-01-26 00:00:00  \N
              5  2210 Bunker Hill Drive      San Mateo    CA               94402     37.5245     -122.344  2017-01-26 00:00:00  \N


Ahora que sabes que los únicos dos concesionarios en California tienen los IDs 2 y 5, respectivamente, puedes entonces consultar la tabla de vendedores, de la siguiente manera:


In [ ]:
sql_exec_query('''
SELECT *
FROM salespeople
WHERE dealership_id in (2, 5)
ORDER BY 1 LIMIT 9;
''')

# Figura 3.4: Vendedores en California

  salesperson_id    dealership_id  title    first_name    last_name    suffix    username       gender    hire_date            termination_date
----------------  ---------------  -------  ------------  -----------  --------  -------------  --------  -------------------  ------------------
              23                2           Beauregard    Peschke                bpeschkem      Male      2021-05-09 00:00:00
              51                5           Lanette       Gerriessen             lgerriessen1e  Female    2021-02-18 00:00:00
              57                5           Spense        Pithcock               spithcock1k    Male      2020-08-11 00:00:00
              61                5           Ludvig        Baynam                 lbaynam1o      Male      2019-04-22 00:00:00
              62                2           Carroll       Pudan                  cpudan1p       Female    2019-01-12 00:00:00
              63                2           Adrianne      Otham                 

Aunque puedes obtener la información que buscas con el método actual, implica hacer dos consultas separadas, lo que puede resultar tedioso. Sería mucho más práctico si pudiéramos fusionar la información de los concesionarios y los vendedores en una sola tabla y luego simplemente filtrar los resultados para mostrar solo los de California. Afortunadamente, SQL nos ofrece una solución eficiente para esto: la cláusula JOIN. Esta cláusula nos permite unir dos o más tablas en una, basándonos en criterios específicos que definimos.

## Tipos de Joins
En este capítulo, aprenderás sobre tres joins fundamentales, que se ilustran en la siguiente figura: joins internos, joins externos y cross joins:

<figure>
<center><img src='https://github.com/limspiga/data-modeling/blob/main/images/7b05c1b2-f698-4eea-8593-cfebb8519a76.png?raw=true' width="300" />
<figcaption>
Figura 3.5: Tipos principales de joins</figcaption></center>
</figure>





### Inner Joins

Un "join interno" es una herramienta que usamos para combinar información de dos tablas diferentes, pero solo cuando cumplen con una condición específica que establecemos, llamada "predicado de join". Esta condición, la mayoría de las veces, es una igualdad, como que un número o nombre en una tabla sea igual a un número o nombre en la otra tabla.

Imagina que es como un filtro que solo deja pasar las combinaciones de filas de ambas tablas que cumplen con esta condición. Si una combinación de filas no cumple con la condición que establecimos, simplemente no la incluimos en los resultados finales.

Para hacer un "join interno", cada fila de la primera tabla se revisa junto con cada fila de la segunda tabla para ver si cumplen con la condición de "join". Solo vemos en los resultados las combinaciones que sí la cumplen.

A continuación, veremos cómo se formula esto técnicamente en SQL:

```sql
SELECT {columns}
FROM {table1}
INNER JOIN {table2}
  ON {table1}.{common_key_1}={table2}.{common_key_2};
```



Aquí, {columns} son las columnas que quieres obtener de la tabla unida, {table1} es la primera tabla, {table2} es la segunda tabla, {common_key_1} es la columna en {table1} a la que quieres unirte, y {common_key_2} es la columna en {table2} a la que quieres unirte.

Ahora, regresa a las dos tablas discutidas anteriormente: concesionarios y vendedores. Como se mencionó anteriormente, sería bueno si pudieras agregar la información de la tabla de concesionarios a la tabla de vendedores sabiendo en qué estado está cada concesionario. Por el momento, supone que todos los ID de los vendedores tienen un valor dealership_id válido.

Puedes unir las dos tablas usando una condición de igualdad en el predicado de unión, como sigue:


In [ ]:
sql_exec_query('''
SELECT *
FROM salespeople
INNER JOIN dealerships
  ON salespeople.dealership_id = dealerships.dealership_id
ORDER BY 1 LIMIT 10;
''')

# Figura 3.6: La tabla de vendedores unida a la tabla de concesionarios

  salesperson_id    dealership_id  title    first_name    last_name    suffix    username    gender    hire_date            termination_date      dealership_id  street_address          city          state      postal_code    latitude    longitude  date_opened          date_closed
----------------  ---------------  -------  ------------  -----------  --------  ----------  --------  -------------------  ------------------  ---------------  ----------------------  ------------  -------  -------------  ----------  -----------  -------------------  -------------
               1               17           Electra       Elleyne                eelleyne0   Female    2020-01-26 00:00:00                                   17  2120 Walnut Street      Philadelphia  PA               19092     39.951      -75.177   2017-01-26 00:00:00  \N
               2                6           Montague      Alcoran                malcoran1   Male      2021-08-27 00:00:00                                    6  731



Como puedes ver en el resultado anterior, la tabla es el resultado de unir la tabla de vendedores con la de concesionarios. Nota que la primera tabla listada en la consulta, vendedores, está en el lado izquierdo del resultado, mientras que la tabla de concesionarios está en el lado derecho. Este orden de izquierda-derecha será muy importante en la próxima sección cuando aprendas sobre los joins externos entre tablas. Durante un join externo, si una tabla está en el lado izquierdo o derecho puede afectar el resultado de la consulta. Para un join interno, sin embargo, el orden de las tablas no es importante para los predicados de unión que usan una operación de igualdad.

Ahora, mira las columnas involucradas; dealership_id en la tabla de vendedores coincide con dealership_id en la tabla de concesionarios. Esto muestra cómo se cumple el predicado de unión. Al ejecutar esta consulta de unión, has creado efectivamente un nuevo "superconjunto de datos" que consta de las dos tablas fusionadas donde las dos columnas dealership_id son iguales.

Ahora puedes ejecutar una consulta SELECT sobre este "superconjunto de datos" de la misma manera que una tabla grande usando las cláusulas y palabras clave del Capítulo 2, Los conceptos básicos de SQL para análisis. Por ejemplo, volviendo al problema de múltiples consultas para determinar cuál de las consultas de ventas funciona en California, ahora puedes abordarlo con una consulta fácil:


In [ ]:
sql_exec_query('''
SELECT *
FROM salespeople
INNER JOIN dealerships
  ON salespeople.dealership_id = dealerships.dealership_id
WHERE dealerships.state = 'CA'
ORDER BY 1 LIMIT 9;
''')

# Figura 3.7: Vendedores en California con una consulta

  salesperson_id    dealership_id  title    first_name    last_name    suffix    username       gender    hire_date            termination_date      dealership_id  street_address              city         state      postal_code    latitude    longitude  date_opened          date_closed
----------------  ---------------  -------  ------------  -----------  --------  -------------  --------  -------------------  ------------------  ---------------  --------------------------  -----------  -------  -------------  ----------  -----------  -------------------  -------------
              23                2           Beauregard    Peschke                bpeschkem      Male      2021-05-09 00:00:00                                    2  808 South Hobart Boulevard  Los Angeles  CA               90005     34.0578     -118.305  2017-01-26 00:00:00  \N
              51                5           Lanette       Gerriessen             lgerriessen1e  Female    2021-02-18 00:00:00                     


Esto te da el siguiente resultado, que muestra las primeras filas del conjunto de resultados completo:


Observarás que el resultado en la Figura 3.6 y la Figura 3.7 es casi idéntico, siendo la excepción que la tabla en la Figura 3.7 tiene los datos de los concesionarios añadidos también. Si quieres recuperar solo la parte de la tabla de vendedores de esto, puedes seleccionar las columnas de vendedores usando la siguiente sintaxis de estrella:


In [ ]:
sql_exec_query('''
SELECT salespeople.*
FROM salespeople
INNER JOIN dealerships
  ON dealerships.dealership_id = salespeople.dealership_id
WHERE dealerships.state = 'CA'
ORDER BY 1 LIMIT 9;
''')

# Figura 3.8: Vendedores en California con SELECT alias de tabla

  salesperson_id    dealership_id  title    first_name    last_name    suffix    username       gender    hire_date            termination_date
----------------  ---------------  -------  ------------  -----------  --------  -------------  --------  -------------------  ------------------
              23                2           Beauregard    Peschke                bpeschkem      Male      2021-05-09 00:00:00
              51                5           Lanette       Gerriessen             lgerriessen1e  Female    2021-02-18 00:00:00
              57                5           Spense        Pithcock               spithcock1k    Male      2020-08-11 00:00:00
              61                5           Ludvig        Baynam                 lbaynam1o      Male      2019-04-22 00:00:00
              62                2           Carroll       Pudan                  cpudan1p       Female    2019-01-12 00:00:00
              63                2           Adrianne      Otham                 


Hay otro atajo que puede ayudar al escribir sentencias con varias cláusulas JOIN. Puedes asignar un alias a los nombres de las tablas para evitar escribir el nombre completo de la tabla cada vez. Simplemente escribe el nombre del alias después de la primera mención de la tabla después de la cláusula JOIN, y podrás ahorrar una cantidad considerable de escritura. Por ejemplo, para la consulta anterior, si quisieras asignar un alias a "salespeople" con "s" y a "dealerships" con "d", podrías escribir la siguiente sentencia:

In [ ]:
sql_exec_query('''
SELECT s.*
FROM salespeople s
INNER JOIN dealerships d
  ON d.dealership_id = s.dealership_id
WHERE d.state = 'CA'
ORDER BY 1
LIMIT 9;
''')

  salesperson_id    dealership_id  title    first_name    last_name    suffix    username       gender    hire_date            termination_date
----------------  ---------------  -------  ------------  -----------  --------  -------------  --------  -------------------  ------------------
              23                2           Beauregard    Peschke                bpeschkem      Male      2021-05-09 00:00:00
              51                5           Lanette       Gerriessen             lgerriessen1e  Female    2021-02-18 00:00:00
              57                5           Spense        Pithcock               spithcock1k    Male      2020-08-11 00:00:00
              61                5           Ludvig        Baynam                 lbaynam1o      Male      2019-04-22 00:00:00
              62                2           Carroll       Pudan                  cpudan1p       Female    2019-01-12 00:00:00
              63                2           Adrianne      Otham                 

Alternatively, you could also put the AS keyword between the table name and alias to make the alias more explicit:

In [ ]:
sql_exec_query('''
SELECT s.*
FROM salespeople AS s
INNER JOIN dealerships AS d
  ON d.dealership_id = s.dealership_id
WHERE d.state = 'CA'
ORDER BY 1
LIMIT 9;
''')

  salesperson_id    dealership_id  title    first_name    last_name    suffix    username       gender    hire_date            termination_date
----------------  ---------------  -------  ------------  -----------  --------  -------------  --------  -------------------  ------------------
              23                2           Beauregard    Peschke                bpeschkem      Male      2021-05-09 00:00:00
              51                5           Lanette       Gerriessen             lgerriessen1e  Female    2021-02-18 00:00:00
              57                5           Spense        Pithcock               spithcock1k    Male      2020-08-11 00:00:00
              61                5           Ludvig        Baynam                 lbaynam1o      Male      2019-04-22 00:00:00
              62                2           Carroll       Pudan                  cpudan1p       Female    2019-01-12 00:00:00
              63                2           Adrianne      Otham                 

Ahora que has cubierto los conceptos básicos de las uniones internas (inner joins), es hora de discutir las uniones externas (outer joins).


### Uniones Externas (Outer Joins)
Como se discutió, las uniones internas solo retornarán filas de las dos tablas cuando se cumpla el predicado de unión para ambas tablas, es decir, cuando ambas tablas tengan filas que puedan satisfacer el predicado de unión. De lo contrario, no se retornarán filas de ninguna de las tablas. Puede ocurrir que, a veces, quieras retornar todas las filas de una de las tablas, incluso si la otra tabla no tiene ninguna fila que cumpla con el predicado de unión. En este caso, dado que no hay ninguna fila que cumpla con el predicado de unión, la segunda tabla no retornará nada más que NULL. La unión externa es un tipo de unión en el que todas las filas de al menos una tabla, si cumplen con la condición WHERE de la consulta, se presentarán después de la operación JOIN.

Las uniones externas pueden clasificarse en tres categorías: uniones externas izquierdas (left outer joins), uniones externas derechas (right outer joins) y uniones externas completas (full outer joins):
- Unión externa izquierda (Left outer join): Las uniones externas izquierdas son aquellas en las que se retornará cada fila de la tabla izquierda (es decir, la tabla mencionada primero en una cláusula de unión). Si no se encuentra una fila de la otra tabla (la tabla derecha), se retorna una fila de NULL de la tabla derecha. Las uniones externas izquierdas se realizan utilizando las palabras clave LEFT OUTER JOIN, seguidas de un predicado de unión. Esto también puede escribirse abreviadamente como LEFT JOIN.

  Para mostrar cómo funcionan las uniones externas izquierdas, examina dos tablas: la tabla de clientes (customers) y la tabla de correos electrónicos (emails). Por el momento, asume que no a todos los clientes se les ha enviado un correo electrónico, y quieres enviar correos a todos los clientes que no han recibido un correo electrónico. Puedes utilizar una unión externa izquierda para hacer que eso ocurra, ya que el lado izquierdo de la unión es la tabla de clientes. Para ayudar a gestionar la salida, limitarás esta a las primeras 1,000 filas. Se utiliza el siguiente fragmento de código:

In [ ]:
sql_exec_query('''
SELECT *
FROM
 customers c
LEFT OUTER JOIN
  emails e ON e.customer_id=c.customer_id
ORDER BY
  c.customer_id
LIMIT
10;
''')

# Figure 3.9: Customers left-joined to emails

  customer_id  title    first_name    last_name    suffix    email                      gender    ip_address     phone         street_address       city         state      postal_code    latitude    longitude  date_added             email_id    customer_id  email_subject                               opened    clicked    bounced    sent_date            opened_date          clicked_date
-------------  -------  ------------  -----------  --------  -------------------------  --------  -------------  ------------  -------------------  -----------  -------  -------------  ----------  -----------  -------------------  ----------  -------------  ------------------------------------------  --------  ---------  ---------  -------------------  -------------------  --------------
            1           Arlena        Riveles                ariveles0@stumbleupon.com  F         98.36.172.246                                                                                                   2019-12-19

Cuando observas el resultado de la consulta, deberías ver que están presentes las entradas de la tabla de clientes. Sin embargo, para algunas de las filas, como la del customer_id 27, que se puede ver en la Figura 3.9, las columnas que pertenecen a la tabla de correos electrónicos están completamente llenas de valores NULL. Esta configuración explica cómo el outer join es diferente del inner join. Si se hubiera utilizado el inner join, la fila del customer_id 27 no aparecería, porque no hay un registro correspondiente en la tabla de correos electrónicos.

Esta consulta, sin embargo, sigue siendo útil porque ahora puedes utilizarla para encontrar a las personas que nunca han recibido un correo electrónico. Debido a que esos clientes a los que nunca se les envió un correo electrónico tienen una columna customer_id nula en los valores devueltos de la tabla de correos electrónicos, puedes encontrar a todos estos clientes verificando la columna customer_id en la tabla de correos electrónicos, de la siguiente manera:

In [ ]:
sql_exec_query('''
SELECT
    c.customer_id,
    c.title,
    c.first_name,
    c.last_name,
    c.suffix,
    c.email,
    e.email_id,
    e.email_subject,
    e.opened,
    e.clicked,
    e.bounced,
    e.sent_date,
    e.opened_date,
    e.clicked_date
FROM
  customers c
LEFT OUTER JOIN
  emails e ON c.customer_id = e.customer_id
WHERE
  e.customer_id IS NULL
ORDER BY
  c.customer_id
LIMIT 1000;
''')

# Figura 3.10: Clientes a los que no se les ha enviado correos electrónicos

Como puedes ver, todas las entradas están en blanco en la columna email_id de la tabla de correos electrónicos, lo que indica que el cliente de esa fila no ha recibido ningún correo electrónico. Simplemente podrías tomar los correos electrónicos de esta unión para obtener a todos los clientes que no han recibido un correo electrónico.

• Unión externa derecha: Una unión externa derecha es muy similar a una unión externa izquierda, excepto que ahora la tabla de la "derecha" (la segunda tabla listada) tendrá todas las filas visibles, y la tabla de la "izquierda" tendrá valores NULL si no se cumple la condición de UNIÓN. Para ilustrar, vamos a "invertir" la última consulta uniendo por la derecha la tabla de correos electrónicos a la tabla de clientes con la siguiente consulta:

In [ ]:
sql_exec_query('''
SELECT
    e.email_id,
    e.email_subject,
    e.opened,
    e.clicked,
    e.bounced,
    e.sent_date,
    e.opened_date,
    e.clicked_date,
    c.customer_id,
    c.title,
    c.first_name,
    c.last_name,
    c.suffix,
c.email
FROM emails e
RIGHT OUTER JOIN customers c
  ON e.customer_id=c.customer_id
ORDER BY
  c.customer_id
LIMIT
1000;
''')

# Figura 3.11: Correos electrónicos unidos por la derecha a la tabla de clientes

Observa que este resultado es similar al que se produjo en la Figura 3.9, excepto que los datos de la tabla de correos electrónicos están ahora en el lado izquierdo, y los datos de la tabla de clientes están en el lado derecho. Una vez más, el customer_id 27 tiene NULL para el correo electrónico. Esto muestra la simetría entre una unión por la derecha y una unión por la izquierda.

Unión externa completa: Finalmente, está la unión externa completa. La unión externa completa devolverá todas las filas de las tablas izquierda y derecha, independientemente de si se cumple el predicado de unión. Para las filas donde se cumple el predicado de unión, las dos filas se combinan tal como en una unión interna. Para las filas donde no se cumple, cada fila de ambas tablas será seleccionada como una fila individual, con NULL llenando las columnas de la otra tabla. La unión externa completa se invoca usando la cláusula FULL OUTER JOIN, seguida de un predicado de unión. Aquí está la sintaxis de esta unión:

In [ ]:
sql_exec_query('''
SELECT *
FROM emails e
FULL OUTER JOIN
  customers c
  ON e.customer_id=c.customer_id;
''')

# Figura 3.12: Los correos electrónicos están completamente unidos por fuera a la tabla de clientes

En esta sección, aprendiste cómo implementar tres tipos diferentes de uniones externas. En la siguiente sección, aprenderás sobre la unión cruzada.

Uniones Cruzadas
La unión cruzada es un tipo de unión que no tiene un predicado de unión. Eso significa que cada fila de la tabla "izquierda" se emparejará con todas las filas de la tabla "derecha", independientemente de si están relacionadas o no. También se le conoce como producto cartesiano. Se le llama "cartesiano" en honor al matemático francés René Descartes, quien planteó la idea de este tipo de operación. Se puede invocar usando la cláusula CROSS JOIN, seguida del nombre de la otra tabla. Para entender mejor esto, tomemos el ejemplo de la tabla de productos.

Un análisis común es el análisis de cesta de mercado, que estudia los patrones de venta entre múltiples productos. Por ejemplo, los pañales se venden usualmente junto con toallitas para bebés. Entonces, si estás organizando un sorteo de dos meses para pañales con fines de marketing y esperas que más clientes vengan al pasillo de pañales o a la página web, podrías querer colocar también toallitas para bebés allí. Para llevar a cabo un análisis de cesta de mercado, quieres conocer todas las posibles combinaciones de dos productos que podrías crear a partir de un conjunto dado de productos (como los que se encuentran en la tabla de productos) para crear un sorteo de dos meses con fines de marketing. Puedes utilizar una unión cruzada para obtener la respuesta a la pregunta utilizando la siguiente consulta:

In [ ]:
sql_exec_query('''
SELECT
  P1.product_id, p1.model,
  P2.product_id, p2.model
FROM
  products p1
CROSS JOIN
  products p2;
''')

# Figure 3.13: The cross join of a product to itself

En este caso, has unido cada valor de cada campo en una tabla con el mismo
en otra tabla. El resultado de la consulta tiene 144 filas, que es el equivalente a multiplicar los 12 productos por los mismos 12 productos (12 * 12). También puedes ver que la unión cruzada no requiere un predicado de unión. En otras palabras, una unión cruzada puede simplemente considerarse como una unión externa sin condiciones para unirse.

En general, las uniones cruzadas no se usan mucho en la práctica ya que pueden entorpecer el proceso si no se manejan con cuidado. Unir cruzadamente dos tablas grandes puede resultar en la creación de cientos de miles de millones de filas, lo que puede estancar y colapsar una base de datos. Por lo tanto, si decides usar una unión cruzada, asegúrate de tomar el máximo cuidado al usarla.

Hasta ahora, has cubierto lo básico del uso de uniones para fusionar tablas para un análisis personalizado de datos. Practicarás esto en el siguiente ejercicio.

## Ejercicio 3.01: Usando Uniones para Analizar una Concesionaria de Ventas
En este ejercicio, utilizarás las uniones para juntar tablas relacionadas. Por ejemplo, el jefe de ventas de tu empresa quisiera una lista de todos los clientes que compraron un coche. Para realizar la tarea, necesitas crear una consulta que devuelva todos los ID de los clientes, nombres, apellidos y números de teléfono válidos de los clientes que compraron un coche.

Para completar este ejercicio, realiza los siguientes pasos:
1. Abre pgAdmin, conéctate a la base de datos sqlda y abre el editor de consultas SQL.
2. Usa una unión interna para juntar las tablas de ventas, clientes y productos, la cual devolverá datos para los ID de los clientes, nombres, apellidos y números de teléfono válidos:

In [ ]:
sql_exec_query('''
SELECT
  c.customer_id, c.first_name,
  c.last_name, c.phone
FROM sales s
INNER JOIN
  customers c ON c.customer_id=s.customer_id
INNER JOIN
  products p ON p.product_id=s.product_id
WHERE
  p.product_type='automobile'
  AND c.phone IS NOT NULL;
''')

# Figure 3.14: Customers who bought a car

Puedes ver que ejecutar la consulta te ayudó a unir los datos de las tablas de ventas, clientes y productos y obtener una lista de clientes que compraron un coche y tienen un número de teléfono.

En este ejercicio, utilizando uniones, pudiste reunir tablas relacionadas de manera fácil y eficiente. Varias veces, también querrás combinar el resultado de tus consultas para formar nuevas consultas, de modo que puedas construir análisis de datos sobre análisis existentes. Ahora puedes avanzar para aprender sobre métodos para unir consultas en un conjunto de datos.

# Subconsultas (Subqueries)
Hasta ahora, has estado extrayendo datos de tablas. Puedes haber observado que los resultados de todas las consultas SELECT son relaciones bidimensionales que se parecen a las tablas en una base de datos relacional. Sabiendo esto, puedes preguntarte si hay alguna forma de usar las relaciones producidas por las consultas SELECT en lugar de referenciar una tabla existente en tu base de datos. La respuesta es "sí". Simplemente puedes tomar una consulta, insertarla entre un par de paréntesis y darle un alias. Esto te ayudará a construir un análisis sobre un análisis existente, reduciendo así los errores y mejorando la eficiencia.

Por ejemplo, si quisieras encontrar a todos los vendedores que trabajan en California y obtener los resultados igual que en la Figura 3.7, podrías escribir la consulta usando la siguiente alternativa:

In [ ]:
sql_exec_query('''
SELECT *
FROM
  salespeople
INNER JOIN (
  SELECT
* FROM
     dealerships
  WHERE
dealerships.state = 'CA' )d
ON d.dealership_id = salespeople.dealership_id
ORDER BY
1;
''')

# Figure 3.14: Customers who bought a car

Aquí, en lugar de unir las dos tablas y filtrar las filas donde el estado es igual a 'CA', primero encuentras los concesionarios donde el estado es igual a 'CA', y luego unes internamente las filas de esa consulta con los vendedores.

Si una consulta solo tiene una columna, puedes usar una subconsulta con la palabra clave IN en una cláusula WHERE. Por ejemplo, otra forma de extraer los detalles de la tabla de vendedores usando el ID del concesionario para el estado de California sería como sigue:

In [ ]:
sql_exec_query('''
SELECT *
FROM
  salespeople
  WHERE dealership_id IN (
  SELECT dealership_id FROM dealerships
  WHERE dealerships.state = 'CA'
  )
ORDER BY 1;
''')


Como se ilustra en todos estos ejemplos, es bastante fácil escribir la misma consulta utilizando múltiples técnicas. En la próxima sección, aprenderás sobre uniones.

# Uniones (Unions)
Hasta ahora, en este capítulo, has aprendido cómo unir datos horizontalmente. Puedes usar uniones para agregar nuevas columnas horizontalmente. Sin embargo, puedes estar interesado en juntar múltiples consultas verticalmente, es decir, manteniendo el mismo número de columnas pero agregando múltiples filas. Por favor, ve este ejemplo para más claridad sobre esto.

Supongamos que quisieras visualizar las direcciones de los concesionarios y los clientes usando Google Maps. Para hacer esto, necesitarías las direcciones de tanto los clientes como los concesionarios. Podrías construir una consulta con todas las direcciones de los clientes de la siguiente manera:

In [ ]:
sql_exec_query('''
SELECT
  street_address, city, state, postal_code
FROM
  customers
WHERE
  street_address IS NOT NULL;
''')


También podrías recuperar las direcciones de los concesionarios con la siguiente consulta:

In [ ]:
sql_exec_query('''
SELECT
  street_address, city, state, postal_code
FROM
  dealerships
WHERE
  street_address IS NOT NULL;
''')

Para reducir la complejidad, sería ideal si hubiera una forma de ensamblar las dos consultas en una lista con una única consulta. Aquí es donde entra en juego la palabra clave UNION. Puedes usar las dos consultas anteriores y crear la siguiente consulta:

In [ ]:
sql_exec_query('''
( SELECT
  street_address, city, state, postal_code
FROM
  customers
WHERE
  street_address IS NOT NULL
)UNION ( SELECT
  street_address, city, state, postal_code
FROM
  dealerships
WHERE
  street_address IS NOT NULL
)
ORDER BY 1;
''')

# Figure 3.15: Union of addresses

Ten en cuenta que existen ciertas condiciones que debes tener en mente cuando usas UNION. En primer lugar, UNION requiere que las subconsultas tengan el mismo número de columnas y los mismos tipos de datos para las columnas. Si no es así, la consulta no se ejecutará. En segundo lugar, técnicamente UNION puede que no devuelva todas las filas de sus subconsultas. UNION, por defecto, elimina todas las filas duplicadas en el resultado. Si deseas conservar las filas duplicadas, es preferible usar la palabra clave UNION ALL. Por ejemplo, si ambas consultas anteriores devuelven una fila con valores de dirección tales como '123 Main St', 'Madison', 'WI', '53710', el resultado de la instrucción UNION solo contendrá un registro para este conjunto de valores, pero el resultado de la instrucción UNION ALL incluirá dos registros del mismo valor, uno de cada consulta.

En el próximo ejercicio, implementarás operaciones de unión.



## Ejercicio 3.02: Generando una Lista de Invitados de Clientes Élite Usando UNION
En este ejercicio, ensamblarás dos consultas usando UNION. Para ayudar a construir conciencia de marketing para el nuevo Modelo Chi, el equipo de marketing quisiera organizar una fiesta para algunos de los clientes más ricos de ZoomZoom en Los Ángeles, CA. Para ayudar a facilitar la fiesta, les gustaría que hicieras una lista de invitados con clientes de ZoomZoom que viven en Los Ángeles, CA, así como vendedores que trabajan en el concesionario ZoomZoom en Los Ángeles, CA. La lista de invitados debería incluir detalles como los nombres y apellidos y si el invitado es un cliente o un empleado.


Para completar la tarea, ejecuta lo siguiente:

1. Abre pgAdmin, conéctate a la base de datos sqlda y abre el editor de consultas SQL.

  Escribe una consulta que haga una lista de los clientes de ZoomZoom y los empleados de la empresa que viven en Los Ángeles, CA. La lista de invitados debe contener nombres y apellidos y si el invitado es un cliente o un empleado:

In [ ]:
sql_exec_query('''
( SELECT
  first_name, last_name, 'Customer' as guest_type
FROM
  customers
WHERE
  city='Los Angeles'
  AND state='CA'
)
UNION (
  SELECT
  first_name, last_name,
  'Employee' as guest_type
FROM
  salespeople s
INNER JOIN
  dealerships d ON d.dealership_id=s.dealership_id
WHERE
  d.city='Los Angeles'
  AND d.state='CA'
);
''')

# Figure 3.16: Customer and employee guest list in Los Angeles, CA

Puedes ver la lista de invitados de clientes y empleados de Los Ángeles, CA, después de ejecutar la consulta UNION.

2. Para demostrar el uso de UNION ALL, primero ejecuta una consulta simple que combine la tabla de productos con todas las filas:

In [ ]:
sql_exec_query('''
SELECT * FROM products
UNION
SELECT * FROM products
ORDER BY 1;
''')

Puedes ver que la consulta devuelve 12 filas y no hay filas duplicadas, justo como la tabla de productos original. Sin embargo, supongamos que ejecutas la siguiente consulta:

In [ ]:
sql_exec_query('''
SELECT * FROM products
UNION ALL
SELECT * FROM products
ORDER BY 1;
''')

Verás que la consulta devuelve 24 filas, en las que cada fila se repite dos veces. Esto se debe a que la instrucción UNION ALL mantiene las filas duplicadas de ambas tablas de productos.

En el ejercicio, usaste la palabra clave UNION para combinar filas de diferentes consultas sin esfuerzo. En la próxima sección, explorarás expresiones de tabla común (CTEs).

## Expresiones de tabla común (CTEs)
Las CTEs son simplemente una versión diferente de las subconsultas. Las CTEs establecen tablas temporales utilizando la cláusula WITH. Para entender mejor esta cláusula, observa la siguiente consulta, que usaste anteriormente para encontrar a los vendedores basados en California:

In [ ]:
sql_exec_query('''
SELECT *
FROM
  salespeople
INNER JOIN (
  SELECT
* FROM
    dealerships
  WHERE
dealerships.state = 'CA' )d
ON d.dealership_id = salespeople.dealership_id
ORDER BY
1;
''')

Esto podría escribirse usando CTEs, de la siguiente manera:

In [ ]:
sql_exec_query('''
WITH d as (
  SELECT
* FROM
    dealerships
  WHERE
    dealerships.state = 'CA'
  )
SELECT *
FROM
  salespeople
INNER JOIN
  d ON d.dealership_id = salespeople.dealership_id
ORDER BY 1;
1;
''')

Una ventaja de las CTEs es que pueden diseñarse para ser recursivas. Las CTEs recursivas pueden referenciarse a sí mismas. Debido a esta característica, puedes usarlas para resolver problemas que otras consultas no pueden. Sin embargo, las CTEs recursivas están más allá del alcance de este libro.
Ahora que conoces varias formas de unir datos a lo largo de una base de datos, observa cómo transformar los datos de estos resultados.

# Limpieza de Datos
A menudo, los datos brutos presentados en el resultado de una consulta pueden no estar en la forma deseada. Es posible que desees eliminar valores, sustituir valores o mapear valores a otros valores. Para lograr estas tareas, SQL proporciona una amplia variedad de declaraciones y funciones. Las funciones son palabras clave que toman entradas (como una columna o un valor escalar) y procesan esas entradas para obtener algún tipo de salida. Aprenderás sobre algunas funciones útiles para la transformación y limpieza de datos en las siguientes secciones.
## La Función CASE WHEN
CASE WHEN es una función que permite que una consulta mapee varios valores en una columna a otros valores. El formato general de una declaración CASE WHEN es el siguiente:



```sql
CASE
  WHEN condition1 THEN value1
  WHEN condition2 THEN value2
  ...
  WHEN conditionX THEN valueX
  ELSE else_value
END;
```



Aquí, condition1 y condition2, hasta conditionX, son condiciones booleanas; valor1 y valor2, hasta valorX, son valores que se asignarán conforme a las condiciones booleanas; y else_value es el valor que se asignará si ninguna de las condiciones booleanas se cumple. Para cada fila, el programa comienza desde la parte superior de la instrucción CASE WHEN y evalúa la primera condición booleana. Luego, el programa recorre cada condición booleana desde la primera. Para la primera condición desde el inicio de la instrucción que se evalúa como verdadera, la instrucción devolverá el valor asociado con esa condición. Si ninguna de las instrucciones se evalúa como verdadera, entonces se devolverá el valor asociado con la instrucción ELSE.

Por ejemplo, quieres devolver todas las filas para los clientes de la tabla de clientes. Además, te gustaría añadir una columna que etiquete a un usuario como siendo de tipo "Cliente Élite" si vive en el código postal 33111, o como tipo "Cliente Premium" si vive en el código postal 33124. De lo contrario, marcará al cliente como tipo "Cliente Estándar". Esta columna se llamará customer_type. Puedes crear esta tabla utilizando una instrucción CASE WHEN, como sigue:

In [ ]:
sql_exec_query('''
SELECT CASE
    WHEN postal_code='33111' THEN 'Elite Customer'
    WHEN postal_code='33124' THEN 'Premium Customer'
      ELSE 'Standard Customer'
  END AS customer_type,
  *
FROM customers LIMIT 12;
''')
# Figure 3.17: The customer_type query

customer_type        customer_id  title    first_name    last_name    suffix    email                    gender    ip_address       phone         street_address    city    state    postal_code    latitude    longitude    date_added
-----------------  -------------  -------  ------------  -----------  --------  -----------------------  --------  ---------------  ------------  ----------------  ------  -------  -------------  ----------  -----------  -------------------
Standard Customer            716           Jarred        Bester                 jbesterjv@nih.gov        M         216.51.110.28                                                                                             2018-09-19 00:00:00
Standard Customer           1228           Ag            Smerdon                asmerdony3@house.gov     F         117.161.100.72                                                                                            2021-12-23 00:00:00
Standard Customer           1876           Gi

Como puedes ver en la tabla anterior, hay una columna llamada customer_type que indica el tipo de cliente que es un usuario. La instrucción CASE WHEN mapeó efectivamente un código postal a una cadena que describe el tipo de cliente. Utilizando una instrucción CASE WHEN, puedes mapear valores de cualquier manera que desees.

## Ejercicio 3.03: Uso de la función CASE WHEN para obtener listas regionales
El objetivo de este ejercicio es crear una consulta que mapee varios valores en una columna a otros valores. Por ejemplo, el jefe de ventas tiene una idea para intentar crear equipos de ventas regionales especializados que puedan vender scooters a los clientes en regiones específicas, en lugar de equipos de ventas genéricos.

Para hacer realidad su idea, el jefe de ventas desea una lista de todos los clientes clasificados por regiones. Para los clientes de los estados de MA, NH, VT, ME, CT o RI, desean que se les etiquete como Nueva Inglaterra. A los clientes de los estados de GA, FL, MS, AL, LA, KY, VA, NC, SC, TN, VI, WV o AR, les gustaría que se les etiquetase como Sureste. Los clientes de cualquier otro estado deben ser etiquetados como Otros.

Para completar este ejercicio, realiza los siguientes pasos:

1. Abre pgAdmin, conéctate a la base de datos sqlda y abre el editor de consultas SQL.
2. Crea una consulta que produzca una columna de customer_id y una columna llamada región, con los estados categorizados como en el siguiente escenario:

In [ ]:
sql_exec_query('''
SELECT
  c.customer_id,
  CASE
    WHEN c.state in (
      'MA', 'NH', 'VT', 'ME',
      'CT', 'RI')
    THEN 'New England'
    WHEN c.state in (
      'GA', 'FL', 'MS',
      'AL', 'LA', 'KY', 'VA',
      'NC', 'SC', 'TN', 'VI',
      'WV', 'AR')
    THEN 'Southeast'
ELSE 'Other'
  END as region
FROM
  customers c
ORDER BY 1 LIMIT 10;

''')
# Figure 3.18: The regional query output

  customer_id  region
-------------  -----------
            1  Other
            2  Other
            3  Southeast
            4  Southeast
            5  Southeast
            6  Southeast
            7  Southeast
            8  New England
            9  Other
           10  Other


Esta consulta asignará un estado a una de las regiones dependiendo de si el estado está en la condición CASE WHEN indicada para esa línea. Deberías obtener el siguiente resultado

En el resultado anterior, en el caso de cada cliente, se ha asignado una región basándose en el estado donde reside el cliente.

En este ejercicio, aprendiste cómo asignar diversos valores en una columna a otros valores utilizando la función CASE WHEN. En la próxima sección, aprenderás sobre una función útil, COALESCE, que ayudará a reemplazar los valores NULL.

## La función COALESCE
Otra necesidad habitual en la manipulación de bases de datos es sustituir los valores NULL por un valor predeterminado. Este objetivo puede alcanzarse de forma sencilla utilizando la función `COALESCE`. Dicha función permite enumerar una serie de columnas y valores escalares; si el primer elemento de la lista es NULL, se reemplaza automáticamente con el segundo valor disponible en la lista. Este proceso continúa iterativamente a lo largo de la lista hasta encontrar un valor que no sea NULL. En caso de que todos los valores de la lista sean NULL, el resultado que retorna la función será NULL.

Para ilustrar un uso simple de la función COALESCE, estudia la tabla de clientes. Algunos de los registros no tienen el valor del campo del teléfono poblado:

<figure>
<center><img src='https://github.com/limspiga/data-modeling/blob/main/images/8a3515a9-c8cc-44c6-ba13-4f6d505209ad.png?raw=true' width="400" />
<figcaption>
Figura 3.19: La consulta COALESCE</figcaption></center>
</figure>


In [ ]:
sql_exec_query('''
SELECT
  first_name, last_name,
  COALESCE(phone, 'NO PHONE') as phone
FROM
customers
ORDER BY 1 LIMIT 10;
''')
# Figura 3.20: La consulta COALESCE

first_name    last_name    phone
------------  -----------  ------------
Aaren         Norrey       NO PHONE
Aaren         Sadat        504-559-3464
Aaren         Whelpdale    607-761-2568
Aaren         Lamlin       414-937-4628
Aaren         Deeman       NO PHONE
Aarika        Emmanuel     NO PHONE
Aarika        Danaher      904-175-3112
Aarika        Guerin       501-121-5841
Aarika        Chadwell     915-856-7492
Aarika        Mawhinney    205-355-4381


Cuando se trata de crear valores predeterminados y evitar NULL, COALESCE siempre será útil.

# La función NULLIF

NULLIF se usa como lo opuesto a COALESCE. Mientras que COALESCE se utiliza para convertir NULL en un valor estándar, NULLIF es una función de dos valores y devolverá NULL si el primer valor es igual al segundo valor.
Por ejemplo, el departamento de marketing ha creado un nuevo material de correo directo para enviar al cliente. Una de las peculiaridades de este nuevo material publicitario es que no puede aceptar personas que tengan títulos (Sr., Dr., Sra., etc.) de más de tres letras. Sin embargo, algunos registros pueden tener un título que es más largo que tres letras. Si el sistema no puede aceptarlos, deben eliminarse durante la recuperación de resultados.
En la base de datos de muestra, el único título conocido que tiene más de tres caracteres es Honorable. Por lo tanto, les gustaría que crees una lista de correo que contenga solo todas las filas con direcciones válidas y que excluya todos los títulos que estén escritos como Honorable sustituyéndolos por NULL. Esto podría hacerse con la siguiente consulta:

In [ ]:
sql_exec_query('''
SELECT customer_id,
        NULLIF(title, 'Honorable') as title,
        first_name,
        last_name,
        suffix,
        email,
        gender,
        ip_address,
        phone,
        street_address,
        city,
        state,
        postal_code,
        latitude,
        longitude,
        date_added
FROM
  customers c
ORDER BY 1;
''')
# Figure 3.21: The NULLIF query

A continuación, aprenderás sobre otros tipos de funciones, como las funciones LEAST y GREATEST.

# Las Funciones MENOR/MAYOR ( LEAST/GREATEST)
Dos funciones que resultan útiles para la preparación de datos son las funciones MENOR y MAYOR. Cada función acepta cualquier cantidad de valores y devuelve el menor o el mayor de los valores, respectivamente.

Por ejemplo, si utilizas la función MENOR con dos parámetros, como 600 y 900, se devolverá 600 como el valor. Esto es lo contrario de lo que devolverá la función MAYOR. Los parámetros pueden ser valores literales o los valores almacenados en campos numéricos.

El uso sencillo de esta variable sería reemplazar el valor si es demasiado alto o bajo. Puedes estudiar un ejemplo detenidamente para entenderlo mejor. Por ejemplo, el equipo de ventas puede querer crear una lista de ventas en la que cada scooter cueste $600 o menos. Puedes crear esto utilizando la siguiente consulta:

In [ ]:
sql_exec_query('''
SELECT
  product_id, model,
  year, product_type,
  LEAST(600.00, base_msrp) as base_msrp,
  production_start_date,
  production_end_date
FROM
  products
WHERE
  product_type='scooter'
ORDER BY 1;
''')
# Figura 3.22: Scooters más económicos

Desde la salida, puedes ver que si base_msrp fuera menor que 600, la consulta SQL devolverá el valor original de base_msrp. Pero si base_msrp es mayor que 600, obtendrás 600 como resultado. Es el valor más bajo entre base_msrp y 600 el que devuelve la consulta, que es lo que se supone que debe hacer la función LEAST().


## La Función de Conversión (Casting)
Otra transformación de datos útil es cambiar el tipo de datos de una columna dentro de una consulta. Esto se hace generalmente para utilizar una función disponible solo para un tipo de datos, como texto, mientras se trabaja con una columna que está en un tipo de datos diferente, como numérico. Para cambiar el tipo de datos de una columna, simplemente necesitas usar el formato columna::tipo_de_datos, donde columna es el nombre de la columna y tipo_de_datos es el tipo de datos al que deseas cambiar la columna.
Por ejemplo, para cambiar el año en la tabla de productos a una columna de texto en una consulta, utiliza la siguiente consulta:

In [ ]:
sql_exec_query('''
SELECT
  product_id,
  model,
  year::TEXT,
  product_type,
  base_msrp,
  production_start_date,
  production_end_date
FROM
  products;
''')
# Figura 3.23: La columna de año como texto

Esto convertirá la columna de año a texto. Ahora puedes aplicar funciones de texto a esta columna transformada. Ten en cuenta que no todos los tipos de datos se pueden convertir a un tipo de datos específico. Por ejemplo, la fecha y hora (datetime) no se puede convertir a tipos de datos de punto flotante (float). Tu cliente de SQL mostrará un error si alguna vez realizas una conversión inesperada.

# Transformación de Datos
Cada conjunto de datos es único, al igual que cada uno de los casos de uso empresarial para los conjuntos de datos. Esto significa que el procesamiento y la transformación de los conjuntos de datos son únicos a su manera. Sin embargo, existen algunas lógicas de procesamiento con las que te encontrarás con frecuencia en el mundo real. Aprenderás algunas de ellas en las secciones de esta sección.

## Las Funciones DISTINCT y DISTINCT ON
Cuando examinas un conjunto de datos, es posible que desees determinar los valores únicos en una columna o grupo de columnas. Este es el caso de uso principal de la palabra clave DISTINCT.

Por ejemplo, si quisieras conocer todos los años de modelo únicos en la tabla de productos, podrías usar la siguiente consulta:

In [ ]:
sql_exec_query('''
SELECT DISTINCT year
FROM products
ORDER BY 1;
''')
# Figure 3.24: Distinct model years

También puedes usarlo con múltiples columnas para obtener todas las combinaciones distintas de columnas presentes. Por ejemplo, para encontrar todos los años de modelo distintos y los tipos de productos que se lanzaron para esos años de modelo, simplemente puedes usar lo siguiente:

In [ ]:
sql_exec_query('''
SELECT DISTINCT year, product_type
FROM products
ORDER BY 1, 2;
''')
# Figure 3.25: Distinct model years and product types

  year  product_type
------  --------------
  2013  scooter
  2014  scooter
  2016  scooter
  2017  automobile
  2017  scooter
  2018  automobile
  2019  scooter
  2020  automobile
  2020  scooter
  2022  automobile
  2022  scooter


Otra palabra clave relacionada con DISTINCT es DISTINCT ON. Ahora, DISTINCT ON te permite asegurarte de que solo se devuelva una fila y que una o más columnas siempre sean únicas en el conjunto. La sintaxis general de una consulta DISTINCT ON es la siguiente:

```sql
SELECT DISTINCT ON (distinct_column)
column_1,
column_2,
...
column_n
FROM table
ORDER BY order_column;
```



Aquí, distinct_column es la(s) columna(s) que deseas que sean distintas en tu consulta, column_1 hasta column_n son las columnas que deseas en la consulta, y order_column te permite determinar la primera fila que se devolverá en una consulta DISTINCT ON si varias columnas tienen el mismo valor para distinct_column.

Para order_column, la primera columna mencionada debe ser distinct_column. Si no se especifica una cláusula ORDER BY, la primera fila se decidirá de manera aleatoria.

Por ejemplo, deseas obtener una lista única de vendedores donde cada vendedor tenga un primer nombre único. En el caso de que dos vendedores tengan el mismo primer nombre, se devolverá el que se unió a la empresa antes. Esta consulta se vería de la siguiente manera:

In [ ]:
sql_exec_query('''
SELECT DISTINCT ON (first_name)
  *
FROM
  salespeople
ORDER BY
  first_name, hire_date;
''')
# Figure 3.26: DISTINCT ON first_name

near "ON": syntax error


Esta tabla garantiza ahora que cada fila tenga un nombre de usuario único. Si hay múltiples usuarios con el mismo primer nombre, la consulta seleccionará al usuario que fue contratado primero por la empresa.

Por ejemplo, si la tabla de vendedores tiene múltiples filas con el nombre Abby, la fila en la Figura 3.26 con el nombre Abby (es decir, la primera fila en los resultados) corresponde a la primera persona contratada en la empresa con el nombre Abby. Del mismo modo, cuando tienes dos empleados con el mismo primer nombre, los resultados de la consulta los ordenarán por la fecha de inicio. Por ejemplo, cuando existen dos empleados, Andrey Haack con la fecha de inicio 2016-01-10 y Andrey Kures con la fecha de inicio 2016-05-17, en la base de datos, Andrey Haack se enumerará primero, ya que su fecha de inicio es anterior.

En la próxima sección, pasarás por una actividad que demuestra cómo SQL se puede utilizar para crear un conjunto de datos para un modelo.

# Actividad 3.01: Construcción de un Modelo de Ventas Utilizando Técnicas de SQL

En esta actividad, limpiarás y prepararás los datos para su análisis utilizando técnicas de SQL. El equipo de ciencia de datos desea construir un nuevo modelo para ayudar a predecir qué clientes son los mejores prospectos para la remarcación. Un nuevo científico de datos se ha unido a su equipo.

Es tu responsabilidad ayudar al nuevo científico de datos a preparar y construir un conjunto de datos que se utilizará para entrenar un modelo. Escribe una consulta para ensamblar un conjunto de datos. Aquí están los pasos a seguir:

1. Utiliza INNER JOIN para unir la tabla de clientes con la tabla de ventas.


In [18]:
sql_exec_query('''
SELECT *
FROM sales s
JOIN customers c
ON s.customer_id = c.customer_id LIMIT 12
''')

  customer_id    product_id  sales_transaction_date      sales_amount  channel    dealership_id      customer_id  title    first_name    last_name    suffix    email                           gender    ip_address       phone         street_address             city          state      postal_code    latitude    longitude  date_added
-------------  ------------  ------------------------  --------------  ---------  ---------------  -------------  -------  ------------  -----------  --------  ------------------------------  --------  ---------------  ------------  -------------------------  ------------  -------  -------------  ----------  -----------  -------------------
        27275             7  2021-03-16 08:40:24              539.991  internet   \N                       27275           Vonni         Bickerstaff            vbickerstaffl1m@scribd.com      F         243.25.198.130   713-436-3552  0868 South Plaza           Houston       TX               77218     29.834      -95.4342  

2. Utiliza INNER JOIN para unir la tabla de productos con la tabla de ventas.


In [19]:
sql_exec_query('''
SELECT *
FROM sales s
JOIN customers c
ON s.customer_id = c.customer_id
JOIN products p
  ON s.product_id = p.product_id LIMIT 12
''')

  customer_id    product_id  sales_transaction_date      sales_amount  channel    dealership_id      customer_id  title    first_name    last_name    suffix    email                           gender    ip_address       phone         street_address             city          state      postal_code    latitude    longitude  date_added             product_id  model                  year  product_type      base_msrp  production_start_date    production_end_date
-------------  ------------  ------------------------  --------------  ---------  ---------------  -------------  -------  ------------  -----------  --------  ------------------------------  --------  ---------------  ------------  -------------------------  ------------  -------  -------------  ----------  -----------  -------------------  ------------  -------------------  ------  --------------  -----------  -----------------------  ---------------------
        27275             7  2021-03-16 08:40:24              539.991  inter

3. Utiliza LEFT JOIN para unir la tabla de concesionarios (tabla derecha) con la tabla de ventas (tabla izquierda).


In [20]:
sql_exec_query('''
SELECT *
FROM sales s
LEFT JOIN dealerships d
  ON d.dealership_id = s.dealership_id
JOIN customers c
  ON s.customer_id = c.customer_id
JOIN products p
  ON s.product_id = p.product_id LIMIT 12
''')

  customer_id    product_id  sales_transaction_date      sales_amount  channel    dealership_id    dealership_id    street_address    city    state    postal_code    latitude    longitude    date_opened    date_closed      customer_id  title    first_name    last_name    suffix    email                           gender    ip_address       phone         street_address             city          state      postal_code    latitude    longitude  date_added             product_id  model                  year  product_type      base_msrp  production_start_date    production_end_date
-------------  ------------  ------------------------  --------------  ---------  ---------------  ---------------  ----------------  ------  -------  -------------  ----------  -----------  -------------  -------------  -------------  -------  ------------  -----------  --------  ------------------------------  --------  ---------------  ------------  -------------------------  ------------  -------  ------------

4. Devuelve todas las columnas de la tabla de clientes y la tabla de productos.



In [24]:
sql_exec_query('''
SELECT
  c.*, p.*
FROM sales s
LEFT JOIN dealerships d
  ON d.dealership_id = s.dealership_id
JOIN customers c
  ON s.customer_id = c.customer_id
JOIN products p
  ON s.product_id = p.product_id
   LIMIT 12;
''')

  customer_id  title    first_name    last_name    suffix    email                           gender    ip_address       phone         street_address             city          state      postal_code    latitude    longitude  date_added             product_id  model                  year  product_type      base_msrp  production_start_date    production_end_date
-------------  -------  ------------  -----------  --------  ------------------------------  --------  ---------------  ------------  -------------------------  ------------  -------  -------------  ----------  -----------  -------------------  ------------  -------------------  ------  --------------  -----------  -----------------------  ---------------------
        27275           Vonni         Bickerstaff            vbickerstaffl1m@scribd.com      F         243.25.198.130   713-436-3552  0868 South Plaza           Houston       TX               77218     29.834      -95.4342  2020-05-12 00:00:00             7  Bat            

5. Devuelve la columna dealership_id de la tabla de ventas, pero llena dealership_id en ventas con -1 si es NULL.


In [25]:
sql_exec_query('''
SELECT
COALESCE(s.dealership_id, -1) sales_dealership,
c.*, p.*
FROM sales s
LEFT JOIN dealerships d
ON d.dealership_id = s.dealership_id
JOIN customers c
ON s.customer_id = c.customer_id
JOIN products p
ON s.product_id = p.product_id
   LIMIT 12;
''')

sales_dealership      customer_id  title    first_name    last_name    suffix    email                           gender    ip_address       phone         street_address             city          state      postal_code    latitude    longitude  date_added             product_id  model                  year  product_type      base_msrp  production_start_date    production_end_date
------------------  -------------  -------  ------------  -----------  --------  ------------------------------  --------  ---------------  ------------  -------------------------  ------------  -------  -------------  ----------  -----------  -------------------  ------------  -------------------  ------  --------------  -----------  -----------------------  ---------------------
\N                          27275           Vonni         Bickerstaff            vbickerstaffl1m@scribd.com      F         243.25.198.130   713-436-3552  0868 South Plaza           Houston       TX               77218     29.834      

6. Agrega una columna llamada "high_savings" que devuelva 1 si el monto de la venta fue de 500 dólares menos que base_msrp o menos. De lo contrario, devuelve 0. Asegúrate de realizar la consulta en una tabla unida.


In [26]:
sql_exec_query('''
SELECT
  COALESCE(s.dealership_id, -1) sales_dealership,
  CASE
WHEN sales_amount < base_msrp - 500 THEN 1
    ELSE 0
  END high_savings,
  c.*, p.*
FROM sales s
LEFT JOIN dealerships d
  ON d.dealership_id = s.dealership_id
JOIN customers c
  ON s.customer_id = c.customer_id
JOIN products p
  ON s.product_id = p.product_id
   LIMIT 12;
''')

sales_dealership      high_savings    customer_id  title    first_name    last_name    suffix    email                           gender    ip_address       phone         street_address             city          state      postal_code    latitude    longitude  date_added             product_id  model                  year  product_type      base_msrp  production_start_date    production_end_date
------------------  --------------  -------------  -------  ------------  -----------  --------  ------------------------------  --------  ---------------  ------------  -------------------------  ------------  -------  -------------  ----------  -----------  -------------------  ------------  -------------------  ------  --------------  -----------  -----------------------  ---------------------
\N                               0          27275           Vonni         Bickerstaff            vbickerstaffl1m@scribd.com      F         243.25.198.130   713-436-3552  0868 South Plaza           Houst


Resultado Esperado:

La siguiente figura muestra algunas de las filas del resultado de esta actividad. Puedes ver que varios dealership_id son reemplazados por -1 en la consulta, ya que en realidad son NULL. Esto se debe a que las ventas por Internet no pasan por un concesionario y, por lo tanto, no tienen un valor de dealership_id. Algunas de las filas también tienen su valor en la columna high_savings marcado como 1, lo que indica que el monto de la venta es de $500 o más por debajo de base_msrp. Puedes revisar algunas filas, intentar obtener los datos originales y confirmar que el SQL esté escrito correctamente:


<figure>
<center><img src='https://github.com/limspiga/data-modeling/blob/main/images/b070c5a4-93d0-4ec6-8a0f-a012406e0617.png?raw=true' width="500" />
<figcaption>
Figure 3.27: Building a sales model query</figcaption></center>
</figure>

# Resumen
SQL te proporciona muchas herramientas para mezclar y limpiar datos. En este capítulo, primero aprendiste cómo combinar dos o más tablas. Comenzaste con la palabra clave JOIN, que fusiona datos de tablas en función de sus columnas comunes. Existen varios tipos de JOIN. Dependiendo de si deseas conservar los datos en una tabla en particular o no, puedes elegir entre INNER JOIN, LEFT OUTER JOIN, RIGHT OUTER JOIN, FULL OUTER JOIN o CROSS JOIN. Luego, aprendiste cómo utilizar subconsultas y CTE (Expresiones de Tabla Común) para preservar y reutilizar los resultados de las consultas. También puedes utilizar UNION y UNION ALL para fusionar los resultados de dos consultas con la misma estructura en un único conjunto de resultados.

Después de aprender cómo combinar datos de diferentes conjuntos de datos, aprendiste a realizar ciertas transformaciones en los datos. Comenzaste con la función CASE WHEN, que es una forma genérica de convertir una expresión en otra basada en condiciones personalizadas definidas. Luego, aprendiste cómo utilizar las funciones COALESCE() y NULLIF() para convertir entre valores NULL y no NULL. También aprendiste cómo cambiar el tipo de datos de una expresión utilizando funciones de conversión, y finalmente, conociste las funciones DISTINCT y DISTINCT ON para obtener listas distintas de valores.

Ahora que sabes cómo preparar un conjunto de datos, en el próximo capítulo aprenderás cómo comenzar a obtener insights analíticos utilizando funciones de agregación.